In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using Device", device)

transform = transforms.ToTensor()

train_data = MNIST(
  root = "data",
  train=True,
  download=True,
  transform=transform
)

test_data = MNIST(
  root = "data",
  train=False,
  download=True,
  transform=transform
)

train_loader = DataLoader(
  train_data,
  batch_size=64,
  shuffle=True
)

test_loader = DataLoader(
  test_data,
  batch_size=64,
  shuffle=False
)

class DigitalModel(nn.Module):
  def __init__(self):
    super().__init__()

    self.net = nn.Sequential(
      nn.Linear(28*28,128),
      nn.ReLU(),

      nn.Linear(128,64),
      nn.ReLU(),

      nn.Linear(64,10)
    )

  def forward(self,x):
    return self.net(x)
  
model = DigitalModel().to(device)\

loss_Fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5;

for epoch in range(epochs):
  model.train()
  total_loss = 0

  for image, labels in train_loader:
    images = image.view(image.size(0),-1).to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_Fn(outputs,labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch+1}/{epoch} | loss: {total_loss:.4f}")

model.eval()
correct = 0
total = 0

with torch.no_grad():
  for image,labels in test_loader:
    images = image.view(image.size(0),-1).to(device)
    labels = labels.to(device)

    outputs = model(images)
    predictions = outputs.argmax(dim=1)

    correct += (predictions == labels).sum().item()
    total += labels.size(0)

accuracy = 100 * correct/total
print(f"Total Accuracy: {accuracy:.2f}%")

torch.save(model.state_dict(),"minst_model.pth")
print("Model Saved as minst_model.pth")

index = 0
image,true_label = test_data[index]

plt.imshow(image.squeeze(), cmap="gray")
plt.title(f"Actual Label: {true_label}")
plt.axis("off")
plt.show()

image_flat = image.view(1,-1).to(device)

with torch.no_grad():
  output = model(image_flat)
  predicted_label = output.argmax(dim=1).item()

print("User picked image index",index)
print("Actual label",true_label)
print("Model Prediction",predicted_label)











Using Device cpu
Epoch 1/0 | loss: 322.4096
Epoch 2/1 | loss: 131.7042
Epoch 3/2 | loss: 90.9461
Epoch 4/3 | loss: 68.8999
Epoch 5/4 | loss: 54.1088
Total Accuracy: 97.37%
Model Saved as minst_model.pth


TypeError: show() got an unexpected keyword argument 'cmap'